In [ ]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr

from tqdm.notebook import tqdm
list1=[100,125,150,175,200]
N=[1]#,0,1,2,3,4]
import warnings
warnings.filterwarnings("ignore")
leverage=10
delta_list=['50','45', '40', '35', '30', '25','20','15','10','5']
def closest_strike(group, input_value):
    return     (np.abs(group['OPT_Close'] - input_value)).argmin()
BN_data=pd.read_csv(r'C:\Projects\Straddles\Data\Monthly/BANKNIFTY-I.csv',dayfirst=True,parse_dates=['Date'])
BN_data['Date']=pd.to_datetime(BN_data['Date'])
direc=r'C:\Projects\Straddles\Data\Monthly\With_contract_filter/'
exp_df = pd.read_csv(r'C:\Projects\Calendar 11-12\data\BN weekly 2016-2022/WeeklyExpiry.csv',parse_dates = ["Weekly_Expiry_Date","date"],dayfirst =True,usecols = ["Weekly_Expiry_Date","date"])
monthly_exp=pd.read_csv(r'C:\Projects\Straddles\Data\Monthly/expiry_dates_monthly.csv',parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date"])
BN_data=pd.read_csv(r'C:\Projects\Straddles\Data\Monthly/BN-1 test.csv',dayfirst=True,parse_dates=['Date'])
BN_data['Date']=pd.to_datetime(BN_data['Date'])

for delta in tqdm(delta_list):
    print(delta)
    idx = pd.read_csv(direc + "/BANKNIFTY-I.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])
    for nnn in N:
            print(nnn)
            
            output_path=r'C:\Projects\Straddles\NEW/Monthly/Base Case'+'/reverse/'+delta+'_delta'+'/'+str(nnn)
            # print(output_path)
            if not os.path.exists(output_path):
                os.makedirs(output_path)

            

            idx_ce = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
            idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")
            final=idx_ce.merge(idx_pe,left_on='Date_CE',right_on='Date_PE')
            final = pd.merge(final, monthly_exp, left_on = 'Date_CE', right_on='Date',how = 'left')
            final = final.T.drop_duplicates().T  #Drop Duplicates Columns
            final.rename(columns = {'Date_CE':'Date'}, inplace = True)

            final.to_csv(output_path+"/check-1data-w3.csv")


            final=pd.read_csv(output_path+"/check-1data-w3.csv",parse_dates=['Date','curr_exp_date'],dayfirst=True)

            df=pd.read_csv(output_path+"/check-1data-w3.csv")
            status=1
            nnn1=nnn
            print(nnn1)
            if nnn>0:
                nnn1=nnn-1
            for idi,row in df.iterrows():
                if nnn1!=4:
                    if idi==0:
                        if nnn!=0:
                            df.loc[idi:idi+nnn1-1,'Status']=0
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,'status']=1
                        else:
                            df.loc[idi,"Marker"]='F'
                            df.loc[idi,'status']=1
                    elif (df.loc[idi-1,'Date'] == df.loc[idi-1,'curr_exp_date']):
                        if nnn!=0:
                            df.loc[idi:idi+nnn-1,"Status"]=0
                        df.loc[idi+nnn1,"Marker"]='F'
                        df.loc[idi+nnn1,"status"]=1
                        df.loc[idi+nnn1,"Marker2"]='F'

                    elif (df.loc[idi,'Date'] == df.loc[idi,'curr_exp_date']):
                        df.loc[idi,"Marker"]='L'
                        df.loc[idi,'status']=1
                elif nnn1==4:
                    if idi==0:
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,'status']=1
                    elif (df.loc[idi,'Date'] == df.loc[idi,'curr_exp_date']):
                        df.loc[idi-1,"Marker"]='F'
                        df.loc[idi,"Marker"]='L'
                        df.loc[idi-1:idi,'status']=1
                    if df.loc[idi,'status']!=1:
                        df.loc[idi,'Status']=0
            
            df.reset_index(drop=True,inplace=True)    
            df.to_csv(output_path+'/dte_test3-3-3-1.csv')
            
            for idi,row in df.iterrows():
                if nnn1!=0 and nnn!=4:
                    if df.loc[idi,'Status']==0:
                        df.drop(idi, axis=0, inplace=True)
                if nnn1==4:
                    if df.loc[idi,'status']!=1:
                        
                        df.drop(idi, axis=0, inplace=True)

            df.to_csv(output_path+'/dte_test3-3-3.csv')
            
            from tqdm.notebook import tqdm
            final=pd.DataFrame()
            dfg=df.groupby(['curr_exp_date'])
            len(dfg)
            temp=0
            for name,group in tqdm(dfg):

                if temp==0:
                    strike_ce=group["Strike_CE"].iat[0]
                    ce_optclose=group["OPT_Close_CE"].iat[0]
                    date=group["Date"].iat[0]
                    strike_pe=group["Strike_PE"].iat[0]

                if temp==1:
                    strike_ce=strike_new_ce
                    ce_optclose=ce_close_new#group["OPT_Close_short"].iat[0]
                    date=group["Date"].iat[0]
                    strike_pe=strike_new_pe
                    pe_optclose=pe_close_new

                strike_ce_w2=strike_ce
                strike_pe_w2=strike_pe
                
               
                x1=x2=x3=y1=y2=y3=0

                temp1=0
                for idi,row in group.iterrows():
                    # if temp==0:
                        date=group.loc[idi,'Date']
                        x_ce=idx[(idx['Date']==date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==strike_ce)]
                        x_pe=idx[(idx['Date']==date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==strike_pe)]
                        if nnn==0:
                            y_ce=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "CE") & (idx2["Strike"]==strike_ce_w2)]
                            y_pe=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "PE") & (idx2["Strike"]==strike_pe_w2)]

                        if len(x_ce):
                            group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                            group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                            group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                            group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                        if len(x_ce)==0:
                                    if len(BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_ce) & (BN_data['Option_Type']=='CE')]):
                                        group.loc[idi,'Strike_CE']=x1=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_ce) & (BN_data['Option_Type']=='CE')]['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_ce) & (BN_data['Option_Type']=='CE')]['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_ce) & (BN_data['Option_Type']=='CE')]['Ticker'].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]
                        if len(x_pe):
                            group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                            group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                            group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                            group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                        if len(x_pe)==0:
                                    if len(BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_pe) & (BN_data['Option_Type']=='PE')]):
                                        group.loc[idi,'Strike_PE']=y1=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_pe) & (BN_data['Option_Type']=='PE')]['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_pe) & (BN_data['Option_Type']=='PE')]['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3=BN_data[(BN_data['Date']==date) & (BN_data['Strike']==strike_pe) & (BN_data['Option_Type']=='PE')]['Ticker'].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                        if row['Marker']=='L':
                            if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                                group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                            else:
                                group.loc[idi,'OPT_Close_PE']=0.05
                            if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                            else:
                                group.loc[idi,'OPT_Close_CE']=0.05
                        if nnn1==0 and nnn==0:
                            if row['Marker']=='L':

                                yy_ce=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "CE")].reset_index(drop=True)
                                strike_new_ce=yy_ce["Delta_"+delta+"_Strike"].iat[0]
                                yy_ce=yy_ce[(yy_ce['Date']==date) & (yy_ce["Option_Type"] == "CE") & (yy_ce["Strike"]==strike_new_ce)]
                                
                                yy_pe=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "PE")].reset_index(drop=True)
                                strike_new_pe=yy_pe["Delta_"+delta+"_Strike"].iat[0]
                                yy_pe=yy_pe[(yy_pe['Date']==date) & (yy_pe["Option_Type"] == "PE") & (yy_pe["Strike"]==strike_new_pe)]
                                
                                group.loc[idi,'Strike_w2_ce']=yy_ce["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_w2_ce']=x2=yy_ce["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_w2_ce']=yy_ce["Ticker"].iat[0]
                                group.loc[idi,'IV_w2_ce']=yy_ce["IV"].iat[0]    # xxx
                                group.loc[idi,'Strike_w2_pe']=yy_pe["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_w2_pe']=y2=yy_pe["OPT_Close"].iat[0]
                                
                                group.loc[idi,'Ticker_w2_pe']=yy_pe["Ticker"].iat[0]
                                group.loc[idi,'IV_w2_pe']=yy_pe["IV"].iat[0] 
                                temp=1
                                temp11=1
                                ce_close_new=group.loc[idi,'OPT_Close_w2_ce']
                                pe_close_new=group.loc[idi,'OPT_Close_w2_pe']

                                if temp==0:
                                    temp=1
                                    continue
         


                final=final.append(group,ignore_index=True)

            final.to_csv(output_path+'/t3-w3-3.csv')
            
            df=pd.read_csv(output_path+'/t3-w3-3.csv')

            df['lotsize']=1
            df['qty']=1

            df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
            df['Margin']=df['Exposure']/leverage


            initial_equity=10000
            df['units'] = (initial_equity/df['Margin']).astype(int)
            df['qty_final'] = df["qty"]*df['units']

            df.to_csv(output_path+"/unit check1=w3.csv")


            df=pd.read_csv(output_path+"/unit check1=w3.csv")
            temp=0
            for idi, row in df.iterrows():
                if idi==0:
                    continue
                else:
                    if nnn>0:
                        df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                        df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                        df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                        if row["Marker"] == "F":
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                    else:
                        if temp==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                        if row["Marker"] == "L" and temp==0:
                            temp=1
                            continue
                        if temp==1:
                            
                            if row["Marker"] == "F" :
                                df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_w2_ce"]
                                df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_w2_pe"]
                                df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_w2_pe"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_w2_ce"])
                            else:
                                df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                 
            df['Total_points']=df['Points'].cumsum()

            df.to_csv(output_path+'/pl_points_'+delta+'.csv')
        
            df=pd.read_csv(output_path+'/pl_points_'+delta+'.csv')
            temp=0
            df['Total_points']=df['Points'].cumsum()
            df['eq_curve']=df['Points']
            df.loc[0,'eq_curve']=25000
            df['eq_curve']=df['eq_curve'].cumsum()
            df.to_csv(output_path+'/eq_curve_'+delta+'.csv')
            